## 채용 사이트 첫번째 페이지만 스크래핑 : excel 저장까지 ~페이징까지도 도전하기
=> Company name | Due Data

In [1]:
from selenium import webdriver

In [2]:
import time

In [3]:
browser = webdriver.Chrome('./chromedriver.exe')

In [4]:
browser.get('https://www.incruit.com/')

#### 이 아래 마크다운은 클릭하려고 만든 마크다운임. 하지만 데이터 전송이 되어야 마지막이 클릭되는데 데이터 전송이 안되서 java 로 해야하뮤ㅠ
browser.find_elements_by_css_selector('a.slct > span')[0].click()

time.sleep(3)
browser.find_elements_by_css_selector('ul > li > a[href*="cd=150"]')[0].click()

time.sleep(3)
sample = browser.find_elements_by_css_selector('button[data-item-val="150_632"]')
browser.execute_script("arguments[0].click();", sample)

In [5]:
browser.get('https://job.incruit.com/jobdb_list/searchjob.asp?occ2=632')

## bs 사용하기

In [6]:
from bs4 import BeautifulSoup

In [7]:
html = BeautifulSoup(browser.page_source, 'html.parser')
html

<html lang="ko"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title> 빅데이터·AI(인공지능) :: 인크루트 채용정보</title>
<meta content=" 빅데이터·AI(인공지능) 인크루트 채용정보(incruit.com) - 믿을 수 있는 취업정보사이트, 경력별, 지역별, 직종별 구인구직정보, 직업별 일자리정보, 실시간 채용정보, 기업별 입사비법" name="Description"/>
<meta content=" 빅데이터·AI(인공지능),취업정보사이트,구인구직사이트,직업,일자리사이트,구인구직사이트,채용정보사이트,인턴,공기업채용,구인광고,구직정보,채용공고,대기업채용정보,여성일자리,기업정보,공사채용,취직,JOB,잡,인크루트,incruit" name="Keywords"/>
<meta content="https://job.incruit.com/jobdb_list/searchjob.asp?occ2=632" property="og:url"/>
<meta content=" 빅데이터·AI(인공지능) :: 인크루트 채용정보" property="og:title"/>
<meta content=" 빅데이터·AI(인공지능) 인크루트 채용정보(incruit.com) - 믿을 수 있는 취업정보사이트, 경력별, 지역별, 직종별 구인구직정보, 직업별 일자리정보, 실시간 채용정보, 기업별 입사비법" property="og:description"/>
<meta content="인크루트(주)" property="og:site_name"/>
<meta content=

### 예시

In [8]:
tags = html.select('div.n_job_list_table_a > table > tbody > tr')
type(tags), len(tags)

(bs4.element.ResultSet, 60)

In [9]:
tag = tags[0]

In [10]:
# Company Name
tag.select('div.companys > div > span > a')[0]['title']

'재단법인경남테크노파크'

In [11]:
# Due Date
tag.select('div.ddays > p')[1].text.strip()

'~07.30 (금)'

In [12]:
uri = 'https://job.incruit.com/jobdb_list/searchjob.asp?occ2=632&page='

results = [] # or list()

for page in range(1,6) :
    target = uri + str(page)
    browser.get(target)
    html = browser.page_source
    
    soup = BeautifulSoup(html, 'html.parser')
    tags = soup.select('div.n_job_list_table_a > table > tbody > tr')
    
    for tag in tags:
        name = tag.select('div.companys > div > span > a')[0]['title']
        if len(tag.select('div.ddays > p')) == 2 :
            date = tag.select('div.ddays > p')[1].text.strip()
        else :
            date = tag.select('div.ddays > p')[0].text.strip()
        results.append([name, date])

len(results)

300

## 엑셀로 저장

In [13]:
import pandas as pd

In [14]:
df = pd.DataFrame(results, columns=['Company_Name','Due_Date'])
df

,Company_Name,Due_Date
0,재단법인경남테크노파크,~07.30 (금)
1,한국산지보전협회,~06.30 (수)
2,(주)더존비즈온,~07.11 (일)
3,재단법인 서울산업진흥원,~07.12 (월)
4,대구경북첨단의료산업진흥재단,~07.12 (월)
...,...,...
295,파워에이치알,채용시
296,파워에이치알,채용시
297,(주)엔씨소프트,~07.15 (목)
298,현대자동차(주),~07.11 (일)


In [16]:
df.to_excel('./saves/incruit_scraping.xls',index=False)